In [1]:
# Add the ETL script folder to path if needed
import os
import sys
sys.path.append("..")  
from dotenv import load_dotenv
load_dotenv()
from src import load_raw_data_to_postgre

from src.load_raw_data_to_postgre import connect_postgres, create_raw_schema_and_table,get_json_files,load_json_files_to_postgres

# ✅ Connect to PostgreSQL
conn = connect_postgres(password= os.getenv("DB_PASSWORD"))  
# 🏗️ Create schema + table if not exists
create_raw_schema_and_table(conn)

# 📂 Load all telegram JSON files from data lake path
json_files = get_json_files("../data/raw/telegram_messages")
print(f"Found {len(json_files)} JSON files.")

# 📤 Insert messages into PostgreSQL
load_json_files_to_postgres(conn, json_files)

# 🔚 Close the connection
conn.close()
print("✅ All messages successfully loaded.")


Found 6 JSON files.


Inserting messages: 100%|██████████| 6/6 [00:00<00:00, 32.01it/s]

✅ All messages successfully loaded.


In [3]:
import pandas as pd
conn = connect_postgres(password=os.getenv("DB_PASSWORD"))

df = pd.read_sql("SELECT id, raw_json , channel , date, file_path FROM raw.telegram_messages LIMIT 5;", conn)
conn.close()

df.head()


C:\Users\tsega\AppData\Local\Temp\ipykernel_4292\2855770815.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SELECT id, raw_json , channel , date, file_path FROM raw.telegram_messages LIMIT 5;", conn)


,id,raw_json,channel,date,file_path
0,97,"{'_': 'Message', 'id': 97, 'out': False, 'date...",CheMed123,2025-07-11,..\data\raw\telegram_messages\2025-07-11\CheMe...
1,96,"{'_': 'Message', 'id': 96, 'out': False, 'date...",CheMed123,2025-07-11,..\data\raw\telegram_messages\2025-07-11\CheMe...
2,95,"{'_': 'Message', 'id': 95, 'out': False, 'date...",CheMed123,2025-07-11,..\data\raw\telegram_messages\2025-07-11\CheMe...
3,94,"{'_': 'Message', 'id': 94, 'out': False, 'date...",CheMed123,2025-07-11,..\data\raw\telegram_messages\2025-07-11\CheMe...
4,93,"{'_': 'Message', 'id': 93, 'out': False, 'date...",CheMed123,2025-07-11,..\data\raw\telegram_messages\2025-07-11\CheMe...


In [4]:
from src.load_telegram_image import connect_postgres, create_images_table, load_image_metadata

conn = connect_postgres(password=os.getenv("DB_PASSWORD"))
create_images_table(conn)
load_image_metadata(conn, base_path="../data/raw/telegram_images")
conn.close()


📸 Found 167 images.


Loading image metadata: 100%|██████████| 167/167 [00:00<00:00, 532.05it/s]


In [5]:
import pandas as pd
import psycopg2
import os

def preview_telegram_images(dbname="my_data_warehouse", user="postgres", password=None, host="localhost", port="5432", limit=10):
    conn = psycopg2.connect(dbname=dbname, user=user, password=password, host=host, port=port)
    query = """
        SELECT channel, date, file_path, width, height, file_size_kb, format, mode
        FROM raw.telegram_images
        ORDER BY date DESC
        LIMIT %s;
    """
    df = pd.read_sql_query(query, conn, params=(limit,))
    conn.close()
    return df

# 🔍 Preview top 10 image records
preview_telegram_images(password=os.getenv("DB_PASSWORD"))


C:\Users\tsega\AppData\Local\Temp\ipykernel_4292\4035582241.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn, params=(limit,))


,channel,date,file_path,width,height,file_size_kb,format,mode
0,CheMed123,2025-07-11,..\data\raw\telegram_images\2025-07-11\CheMed1...,1080,1080,33.753906,JPEG,RGB
1,CheMed123,2025-07-11,..\data\raw\telegram_images\2025-07-11\CheMed1...,940,788,48.751953,JPEG,RGB
2,CheMed123,2025-07-11,..\data\raw\telegram_images\2025-07-11\CheMed1...,1080,1080,139.281250,JPEG,RGB
3,CheMed123,2025-07-11,..\data\raw\telegram_images\2025-07-11\CheMed1...,1080,1080,127.361330,JPEG,RGB
4,CheMed123,2025-07-11,..\data\raw\telegram_images\2025-07-11\CheMed1...,1280,1280,225.533200,JPEG,RGB
5,CheMed123,2025-07-11,..\data\raw\telegram_images\2025-07-11\CheMed1...,904,1280,134.482420,JPEG,RGB
6,CheMed123,2025-07-11,..\data\raw\telegram_images\2025-07-11\CheMed1...,1080,1080,87.725586,JPEG,RGB
7,CheMed123,2025-07-11,..\data\raw\telegram_images\2025-07-11\CheMed1...,905,1280,118.668945,JPEG,RGB
8,CheMed123,2025-07-11,..\data\raw\telegram_images\2025-07-11\CheMed1...,940,788,99.533200,JPEG,RGB
9,CheMed123,2025-07-11,..\data\raw\telegram_images\2025-07-11\CheMed1...,940,788,48.751953,JPEG,RGB


In [ ]:
import os
import sys
sys.path.append("..")  
from dotenv import load_dotenv
load_dotenv()

import psycopg2
from pathlib import Path

# Define path to your SQL file
sql_file_path = Path("../telegram_warehouse/models/staging/stg_telegram_images.py")

# Read the SQL file content
with open(sql_file_path, 'r', encoding='utf-8') as file:
    sql = file.read()

# Connect to your PostgreSQL database
conn = psycopg2.connect(
    dbname="my_data_warehouse",
    user="postgres",
    password=os.getenv("DB_PASSWORD"),
    host="localhost",
    port="5432"
)
cur = conn.cursor()

# Execute the SQL script
cur.execute(sql)

# Commit changes and close connection
conn.commit()
cur.close()
conn.close()


from telegram_warehouse.staging.stg_telegram_image import refresh_staging_telegram_images
import psycopg  # If you're using psycopg (new binary package)

# Connect to your PostgreSQL DB
conn = psycopg.connect(
    dbname="my_data_warehouse",
    user="postgres",
    password="your_password",
    host="localhost",
    port="5432"
)

# ✅ Call your staging refresh function
refresh_staging_telegram_images(conn)

# ✅ Done
conn.close()



SyntaxError: syntax error at or near "def"
LINE 2: def create_staging_telegram_images_table(conn):
        ^
